In [82]:
import pandas as pd
import numpy as np
import plotly.express as px

In [69]:
portfolio = pd.read_csv('/Users/cristobalcordova/Downloads/loans-2021-12-02.csv') # Carga .csv de admin
portfolio = portfolio[['Folio','Principal a pagar','Fecha de Creación','Colateral restante','Colateral inicial','Original liquid principal amount cents','Accrued interest amount cents','Paid interest amount cents']].set_index('Folio')

In [70]:
portfolio['Principal a pagar'] = portfolio['Principal a pagar'] / 100
portfolio['Original liquid principal amount cents'] = portfolio['Original liquid principal amount cents'] / 100
portfolio['Accrued interest amount cents'] = portfolio['Accrued interest amount cents'] / 100
portfolio['Paid interest amount cents'] = portfolio['Paid interest amount cents'] / 100
portfolio['Colateral restante'] = portfolio['Colateral restante'] / 10**8
portfolio['Colateral inicial'] = portfolio['Colateral inicial'] / 10**8


In [71]:
portfolio['Precio inicial credito'] = portfolio['Original liquid principal amount cents'] / (portfolio['Colateral inicial'] * .5)
portfolio['Precio MarginCall'] = portfolio['Principal a pagar'] / (portfolio['Colateral restante'] * .7)
portfolio['Precio liquidacion'] = portfolio['Principal a pagar'] / (portfolio['Colateral restante'] * .8)

In [72]:
# precio ponderado de liquidación
(portfolio["Precio liquidacion"] * portfolio["Colateral restante"]).sum() / portfolio["Colateral restante"].sum()

25799876.587765798

In [73]:
# Colateral custodiado
portfolio['Colateral restante'].sum()

0.61057966

In [62]:
class cartera():

    def __init__(self) -> None:
        
        self.creditos = pd.DataFrame(columns=['Principal a pagar', 'Colateral restante'])

    def add_credito(self, _prestamo, _colateral):

        self.creditos = self.creditos.append(pd.DataFrame([[_prestamo, _colateral]], columns=['Prestamo', 'Colateral']), ignore_index=True)

    def add_creditos(self, df):

        self.creditos = self.creditos.append(df)

    def ltv(self, price):
        self.creditos['LTV'] = self.creditos['Principal a pagar'] / (self.creditos['Colateral restante'] * price)

    def simulacion_crash(self, p, price):
        precio_crash = price * (1 - p) # price debe ser input
        df_sim = self.creditos
        # calculo nuevo ltv con caide del p%
        df_sim['LTV_fin'] = df_sim['Principal a pagar']/(df_sim['Colateral restante'] * price * (1 - p))
        df_sim = df_sim[df_sim['LTV_fin']>= .8]

        # principal_tarjet: 
        # amount: lo que tengo q liquidar
        df_sim['principal_target'] = (0.65 * df_sim['Colateral restante'] * precio_crash)
        df_sim['amount'] = (df_sim['Principal a pagar'] - df_sim['principal_target']) / (1-0.65)
        # df_sim['amount'] = df_sim[['Prestamo', 'amount']].apply(lambda x: x['Prestamo'] if x['Prestamo']<x['amount'] else x['amount'], axis=1)

        # como quedan los creditos despues de la liquidación
        # df_sim['principal_target'].sum()
        df_sim['Prestamo_after_liq'] = (df_sim['Principal a pagar'] - df_sim['amount'])
        df_sim['LTV_after_liq'] = (df_sim['Principal a pagar'] - df_sim['amount']) / (df_sim['Colateral restante'] * precio_crash -df_sim['amount'])
        self.cart_crash = df_sim
        self.cartera_to_liquidate = round(df_sim['amount'].sum())

In [91]:
df = portfolio[['Principal a pagar', 'Colateral restante']]

In [92]:
# creo cartera con créditos
cart = cartera()
cart.add_creditos(df)
cart.ltv(48000000)
prestado = cart.creditos['Principal a pagar'].sum()

In [93]:
simulaciones = pd.DataFrame(columns= ['Promedio prestado', 'Colateral liquidado', 'Caida'])
precio_sim = 48000000
for p in range(20, 36, 1):
    cart.simulacion_crash(p/100, precio_sim)
    liquidado = cart.cart_crash['amount'].sum()
    simulaciones = simulaciones.append(pd.DataFrame(np.array([[prestado,liquidado,p]]),columns= ['Promedio prestado', 'Colateral liquidado', 'Caida']))

/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_53289/3486115452.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sim['principal_target'] = (0.65 * df_sim['Colateral restante'] * precio_crash)
/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_53289/3486115452.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sim['amount'] = (df_sim['Principal a pagar'] - df_sim['principal_target']) / (1-0.65)
/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_53289/3486115452.py:33: 

In [100]:

# simulaciones
simulaciones['Relacion liquidado/prestado'] = simulaciones['Colateral liquidado'] / simulaciones['Promedio prestado']
px.line(simulaciones, x='Caida', y='Relacion liquidado/prestado')